## CPS_ASEC

In [2]:
import sys
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

path = '/home/brian/Documents/ASEC/data/'

### Number of people removed from poverty by social security

https://www2.census.gov/programs-surveys/demo/tables/p60/280/tableB-8.xlsx

In [228]:
# Child Support Data retrieved manually from table B8
cs = pd.Series({'adults': 0.209, 'elderly': .005, 
                'kids': 0.338, 'Total': 0.551})

In [229]:
year = 2022
write_txt(text_dir / 'poverty_ltyear.txt', f'{year}')

# Extract person records for selected variables
variables = ['H_SEQ', 'PPPOS', 
             'PRECORD', 'A_AGE', 'MARSUPWT', 'PERLIS', 
             'SPM_Resources', 'SPM_PovThreshold', 'WKSWORK',
             'PRDISFLG', 'RSNNOTW', 'PYRSN', 'LKWEEKS',
             'SPM_Weight', 'SPM_ID', 'SPM_CapWkCCXpns',
             'SPM_Totval', 'SPM_MedXpns', 'SPM_ChildSupPd',
             'SPM_Poor', 'SPM_NumPer', 'SPM_EIP',
             'SPM_FedTax', 'SPM_FedTaxBC', 'SPM_ACTC',
             'SPM_FICA', 'SPM_EITC', 'SPM_SNAPSub',
             'SPM_SchLunch', 'SPM_WICval', 'SPM_EngVal', 
             'SPM_BBSUBVAL', 'SPM_CapHouseSub', 
             'SPM_ChildcareXpns', 'SPM_WkXpns', 'CSP_VAL']

benefits = ['SS_VAL', 'SSI_VAL', 'UC_VAL', 'VET_VAL', 
            'WC_VAL', 'PAW_VAL', 'CSP_VAL']

dictfile = f'{path}/persfmt23.txt'
dd = open(dictfile).read()
p = f'({"|".join(variables + benefits)})\s+(\d+)\s+(\d+)\s'
cols = {name: (int(start) - 1, int(start) - 1 + int(length)) 
        for name, length, start in re.findall(p, dd)}

datafile = f'{path}/asec2023_pubuse.dat'
df = (pd.read_fwf(datafile, 
                  colspecs=list(cols.values()), 
                  header=None, 
                  names=list(cols.keys()))
        .query('PRECORD == 3'))

# Special age groups
age_grps = lambda x: np.where(x.A_AGE < 19, 'kids', 
                              np.where((x.A_AGE > 17) & (x.A_AGE < 65), 'adults', 
                                       'elderly'))
df = df.assign(AGE_GRP = age_grps)

In [252]:
# Refundable tax credits
df['SPM_RTC'] = df['SPM_ACTC'] + df['SPM_EITC']
df['SPM_Util'] = df['SPM_EngVal'] + df['SPM_BBSUBVAL']
df['SPM_SNAPSL'] = df['SPM_SNAPSub'] + df['SPM_SchLunch']

# Values for cash programs
p = ['SS', 'SSI', 'UC', 'CSP', 'PAW']
for i in p:
    df[f'SPM_{i}'] = df.groupby('SPM_ID')[f'{i}_VAL'].transform('sum')
    
# Total poverty
povtot = df.groupby('SPM_Poor').MARSUPWT.sum()[1.0] / 100
povgrp = df.groupby(['SPM_Poor', 'AGE_GRP']).MARSUPWT.sum()[1.0] / 100

# Calculate effect of programs
p2 = ['RTC', 'CapHouseSub', 'WICval', 'SNAPSub', 
      'SchLunch', 'EngVal', 'BBSUBVAL', 'Util', 'SNAPSL']
res = pd.Series(dtype='float')
res2 = pd.DataFrame()
for i in p + p2:
    df[f'SPM_{i}_Poor'] = ((df['SPM_Resources'] - df[f'SPM_{i}'])
                     < df['SPM_PovThreshold'])*1
    povc = df.groupby(f'SPM_{i}_Poor').MARSUPWT.sum()[1.0] / 100
    povg = df.groupby([f'SPM_{i}_Poor', 'AGE_GRP']).MARSUPWT.sum()[1.0] / 100
    res[i] = (povc - povtot) / 1_000_000
    res2[i] = (povg - povgrp) / 1_000_000
    
# Additions
s = ['FICA', 'ChildcareXpns', 'MedXpns', 'WkXpns', 'FedTaxBC', 
      'ChildSupPd']

for i in s:
    df[f'SPM_{i}_Poor'] = ((df['SPM_Resources'] + df[f'SPM_{i}'])
                     < df['SPM_PovThreshold'])*1
    povc = df.groupby(f'SPM_{i}_Poor').MARSUPWT.sum()[1.0] / 100
    povg = df.groupby([f'SPM_{i}_Poor', 'AGE_GRP']).MARSUPWT.sum()[1.0] / 100
    res[i] = (povc - povtot) / 1_000_000
    res2[i] = (povg - povgrp) / 1_000_000
    
# Manually add child support
tbl = res2.T
tbl['Total'] = res
tbl.loc['CS'] = cs
tbl = -tbl

# Rename
d = {'SS': 'Social Security', 'SSI': 'SSI', 'UC': 
     'Unemployment Insurance', 'Util': 'Utilities Assistance', 
     'RTC': 'Refundable Tax Credits', 'SNAPSub': 'SNAP',
     'CapHouseSub': 'Housing Subsidies', 
     'PAW': 'TANF/General Assistance', 'SchLunch': 'School Lunch',
     'CS': 'Child Support Received',
     'WICval': 'WIC', 'ChildSupPd': 'Child Support Paid',
     'FICA': 'FICA', 'FedTaxBC': 'Federal Income Tax',
     'WkXpns': 'Work Expenses', 'MedXpns': 'Medical Expenses'}

tbl = tbl.loc[d.keys()].rename(d).sort_values(by='Total', ascending=False)
tbl.index.name = ''
tbl['label'] = tbl['Total'].apply('{:.1f}'.format)
tbl['empty'] = [0.01 if tbl.loc[i, 'Total'] < 0 else -0.01 for i in tbl.index] 
tbl.to_csv(data_dir / 'spmtbl22.csv', index_label='name')

# Social Security Poverty Sentence
ss_effect = -tbl.loc['Social Security', 'Total']
text = ('Social Security is by far the main anti-poverty program '+
        f'in the US, removing {ss_effect:.1f} million '+
        f'people from poverty in {year}.')
write_txt(text_dir / 'ss_pov_sentence.txt', text)
print(text)

Social Security is by far the main anti-poverty program in the US, removing 28.9 million people from poverty in 2022.


In [269]:
rtc_effect = -tbl.loc['Refundable Tax Credits', 'Total']
rtc_kids = -tbl.loc['Refundable Tax Credits', 'kids']
snap_effect = -tbl.loc['SNAP', 'Total']
sl_effect = -tbl.loc['School Lunch', 'Total']
tanf_effect = (-tbl.loc['TANF/General Assistance', 'Total']
               * 1_000_000).round(-3)
mx_effect = tbl.loc['Medical Expenses', 'Total']
fica_effect = tbl.loc['FICA', 'Total']
wx_effect = tbl.loc['Work Expenses', 'Total']
text = (f'In {year}, Social Security payments lift income above the '+
        f'poverty line for {ss_effect:.1f} million people, by far the '+
        'most effective program for reducing poverty.\n\nRefundable tax '+
        'credits, which include the refundable portion of the child '+
        'tax credit and the earned income tax credit, remove '+
        f'{rtc_effect:.1f} million people from poverty, including '+
        f'{rtc_kids:.1f} million children. Supplemental nutrition '+
        f'assistance (SNAP) removes {snap_effect:.1f} million people from '+
        f'poverty, while school lunch programs remove {sl_effect:.1f} '+
        'million. Public assistance and welfare programs take '+
        f'{tanf_effect:,.0f} people out of poverty.\n\n Several elements '+
        'add to the number of people in poverty. Medical expenses are the '+
        f'most significant, and push {mx_effect:.1f} million people into '+
        'poverty line. Federal payroll taxes for Social Security and Medicare '+
        f'put {fica_effect:.1f} million people in poverty. Work expenses '+
        f'additionally put {wx_effect:.1f} million people in poverty.')
write_txt(text_dir / 'poverty_elements.txt', text)
print(text)

In 2022, Social Security payments lift income above the poverty line for 28.9 million people, by far the most effective program for reducing poverty.

Refundable tax credits, which include the refundable portion of the child tax credit and the earned income tax credit, remove 6.4 million people from poverty, including 3.6 million children. Supplemental nutrition assistance (SNAP) removes 3.7 million people from poverty, while school lunch programs remove 1.5 million. Public assistance and welfare programs take 566,000 people out of poverty.

 Several elements add to the number of people in poverty. Medical expenses are the most significant, and push 7.1 million people into poverty line. Federal payroll taxes for Social Security and Medicare put 4.1 million people in poverty. Work expenses additionally put 3.3 million people in poverty.


### Distribution of Personal Income

Single person poverty threshold manually retrieved from HHS

In [2]:
year = '2023'
yr = int(year) - 1 # CPS ASEC references prior year
write_txt(text_dir / 'asec_year.txt', f'{yr}')
datafile = f'asec{year}_pubuse.dat'
# Retrieve column info from dictionary
variables = ['PRECORD', 'MARSUPWT', 'PEARNVAL', 'ERN_VAL', 'PTOTVAL']
dictfile = f'persfmt{year[2:]}.txt'
dd = open(path + dictfile).read()
p = f'({"|".join(variables)})\s+(\d+)\s+(\d+)\s'
cols = {name: (int(start) - 1, int(start) - 1 + int(length)) 
        for name, length, start in re.findall(p, dd)}

# Read raw fwf file
df = (pd.read_fwf(path + datafile, 
                  colspecs=list(cols.values()), 
                  header=None, 
                  names=list(cols.keys()))
        .astype({'MARSUPWT': 'int'})
        .query('PRECORD == 3 and MARSUPWT > 0'))

res = {}
for val in ['PTOTVAL', 'PEARNVAL']:
    res[val] = {}
    for i in range(0,99):
        bins=np.arange(-500, 350000, 500)
        cdf = (df.groupby(pd.cut(df[val], bins))
                 .MARSUPWT.sum().cumsum() / df.MARSUPWT.sum())
        res[val][i] = np.interp(i/100, cdf, bins[1:])

data = pd.DataFrame(res) / 1_000
data['Diff'] = data['PTOTVAL'] - data['PEARNVAL']
data.to_csv(data_dir / 'pearn_dist.csv', index_label='percentile')

In [3]:
data = pd.read_csv(data_dir / 'pearn_dist.csv', index_col='percentile')
data.loc[99] = data.loc[98]
data.index = data.index + 1
threshold = 13590
lab = 100 - data[data['PEARNVAL'] > 0].index[0]
labp = 100 - data[data['PEARNVAL'] > (threshold / 1000)].index[0]
tot = 100 - data[data['PTOTVAL'] > 0].index[0]
color = {'PEARNVAL': 'cyan!50!white', 'Diff': 'violet!50!purple'}
sh150 = (df.loc[df.PTOTVAL > 150000, 'MARSUPWT'].sum() / 
         df['MARSUPWT'].sum()) * 100
text = (f'Over calendar year {yr}, {lab} percent of people have '+
        f'any labor income {c_box(color["PEARNVAL"])}. Only '+
        f'{labp} percent of people have labor income above the '+
        f'single-person poverty threshold of \${threshold:,.0f}.'+
        '\n\nTotal income, which includes after-tax labor income plus '+
        f'welfare and capital income, {c_box(color["Diff"])} '+
        f'reaches {tot} percent of people in {yr}. People who '+
        'did not receive any income by the total income measure '+
        'typically live with people who receive income. \n\nIn '+
        f'{yr}, {sh150:.1f} percent of people have total income '+
        'of more than \$150,000. Note that the chart cuts off '+
        'income above \$150,000. ')
write_txt(text_dir / 'pearn.txt', text)
print(text)

Over calendar year 2022, 50 percent of people have any labor income (see \cbox{cyan!50!white}). Only 44 percent of people have labor income above the single-person poverty threshold of \$13,590.

Total income, which includes after-tax labor income plus welfare and capital income, (see \cbox{violet!50!purple}) reaches 71 percent of people in 2022. People who did not receive any income by the total income measure typically live with people who receive income. 

In 2022, 4.9 percent of people have total income of more than \$150,000. Note that the chart cuts off income above \$150,000. 


In [4]:
# Mark meadian worker
wmed = 100 - (data[data['PEARNVAL'] > 0].index[0] / 2)
wmedt = wmed - 11
text = (f'\draw [black!60] (axis cs:{{{wmed}}},\pgfkeysvalueof{{/pgfplots/ymin}}) -- '+
 f'(axis cs:{{{wmed}}}, \pgfkeysvalueof{{/pgfplots/ymax}});\n'+
 f'\\absnode{{{wmedt}}}{{79}}{{\small \color{{black!75}} median\\\ \small \color{{black!75}} worker}}')
write_txt(text_dir / 'pearn_worker_med.txt', text)

### Real Median Household Income

In [5]:
url = ('https://www2.census.gov/programs-surveys/'+
       'demo/tables/p60/279/tableA2.xlsx')
df = pd.read_excel(url, header=4, index_col=0, skipfooter=34)
# Data covering all races
data = df.loc[:'WHITE ALONE 25'].dropna()
data.index = (pd.to_datetime(data.index.astype('str').str[:4]) + 
               pd.DateOffset(months=6))
res = data[['Estimate', 'Estimate.1']]
res = res.rename({'Estimate': 'Median', 
                  'Estimate.1': 'Mean'}, axis=1)
final = pd.DataFrame()
for col in res.columns:
    i = res[col]
    loc1 = np.where(i.index.get_loc('2017-07-01'))[0][1]
    loc2 = np.where(i.index.get_loc('2013-07-01'))[0][1]
    final = final.join(i.iloc[:loc1].rename(f'{col}_LT'), how='outer')
    final = final.join(i.iloc[loc1: loc2].rename(f'{col}_PR'), how='outer')
    final = final.join(i.iloc[loc2:].rename(f'{col}_PR2'), how='outer')
    
final.loc['1989':].divide(1000).to_csv(data_dir / 'real_hh_inc.csv', 
                                     index_label='date')

# Decile thresholds
url = 'https://www2.census.gov/programs-surveys/demo/tables/p60/279/tableA4.xlsx'
df = pd.read_excel(url, header=5, index_col=0, skipfooter=25)
data = df[['10th percentile limit', '90th percentile limit']]
data.index = (pd.to_datetime(data.index.astype('str').str[:4]) + 
               pd.DateOffset(months=6))
res = data.rename({'10th percentile limit': 'p10', 
                   '90th percentile limit': 'p90'}, axis=1)
final = pd.DataFrame()
for col in res.columns:
    i = res[col]
    loc1 = np.where(i.index.get_loc('2017-07-01'))[0][1]
    loc2 = np.where(i.index.get_loc('2013-07-01'))[0][1]
    final = final.join(i.iloc[:loc1].rename(f'{col}_LT'), how='outer')
    final = final.join(i.iloc[loc1: loc2].rename(f'{col}_PR'), how='outer')
    final = final.join(i.iloc[loc2:].rename(f'{col}_PR2'), how='outer')
    
final.loc['1989':].divide(1000).to_csv(data_dir / 'real_hh_inc_dec.csv', 
                                     index_label='date')

In [6]:
df = pd.read_csv(data_dir / 'real_hh_inc.csv', parse_dates=True,
                 index_col='date') * 1000
clmean = c_line('cyan')
ltdt = dtxt(df.index[-1])['year']
prdt = dtxt(df.index[-2])['year']
prdt2 = dtxt(df.index[-3])['year']
vallt = df['Mean_LT'].iloc[-1]
valpr = df['Mean_LT'].iloc[-2]
valpr2 = df['Mean_LT'].iloc[-3]
val00 = df.loc['2000', 'Mean_PR2'].iloc[0]

text = (f'The mean or average household income is \${vallt:,.0f} in '+
        f'{ltdt} {clmean}, compared to \${valpr:,.0f} in {prdt}, and '+
        f'\${valpr2:,.0f} in {prdt2}. In 2000, real mean household '+
        f'income was \${val00:,.0f}. ')
write_txt(text_dir / 'hh_inc1.txt', text)
print(text, '\n')

clmed = c_line('violet')
valmedlt = df['Median_LT'].iloc[-1]
valmedpr = df['Median_LT'].iloc[-2]
valmedpr2 = df['Median_LT'].iloc[-3]
valmed00 = df.loc['2000', 'Median_PR2'].iloc[0]
medch = ((df['Median_LT'].iloc[-1] / 
          df.loc['2000', 'Median_PR2'].iloc[0]) - 1) * 100
medincdec = value_text(medch, adj='total')

text = (f'Real median household income {clmed}, the price-adjusted '+
        f'midpoint among household incomes, is \${valmedlt:,.0f} in '+
        f'{ltdt}, \${valmedpr:,.0f} in {prdt}, and \${valmedpr2:,.0f} in '+
        f'{prdt2}. For comparison, real median household income was '+
        f'\${valmed00:,.0f} in 2000. Since 2000, real median income '+
        f'{medincdec}.')
write_txt(text_dir / 'hh_inc2.txt', text)
print(text, '\n')

df = pd.read_csv(data_dir / 'real_hh_inc_dec.csv', parse_dates=True,
                 index_col='date') * 1000
cl90 = c_line('teal')
cl10 = c_line('orange!80!yellow')
val90lt = df['p90_LT'].iloc[-1]
val90pr = df['p90_LT'].iloc[-2]
val90pr2 = df['p90_LT'].iloc[-3]
val9000 = df.loc['2000', 'p90_PR2'].iloc[0]
val10lt = df['p10_LT'].iloc[-1]
val10pr = df['p10_LT'].iloc[-2]
val10pr2 = df['p10_LT'].iloc[-3]
val1000 = df.loc['2000', 'p10_PR2'].iloc[0]

text = ('The price-adjusted income limit for the 90th percentile '+
        f'is \${val90lt:,.0f} in {ltdt} {cl90}, \${val90pr:,.0f} in {prdt}, '+
        f'\${val90pr2:,.0f} in {prdt2}, and \${val9000:,.0f} in 2000. Ten '+
        'percent of households make more than this level. \n\n'+
        'On the opposite end of the income distribution, the 10th '+
        f'percentile income limit is \${val10lt:,.0f} in {ltdt} {cl10}, '+
        f'\${val10pr:,.0f} in {prdt}, \${val10pr2:,.0f} in {prdt2}, and '+
        f'\${val1000:,.0f} in 2000. Ten percent of households make '+
        'less than this level. ')
write_txt(text_dir / 'hh_inc3.txt', text)
print(text)

The mean or average household income is \$106,400 in 2022 (see {\color{cyan}\textbf{---}}), compared to \$110,300 in 2021, and \$109,900 in 2020. In 2000, real mean household income was \$91,810.  

Real median household income (see {\color{violet}\textbf{---}}), the price-adjusted midpoint among household incomes, is \$74,580 in 2022, \$76,330 in 2021, and \$76,660 in 2020. For comparison, real median household income was \$67,470 in 2000. Since 2000, real median income increased by a total of 10.5 percent. 

The price-adjusted income limit for the 90th percentile is \$216,000 in 2022 (see {\color{teal}\textbf{---}}), \$228,600 in 2021, \$227,700 in 2020, and \$180,000 in 2000. Ten percent of households make more than this level. 

On the opposite end of the income distribution, the 10th percentile income limit is \$17,100 in 2022 (see {\color{orange!80!yellow}\textbf{---}}), \$16,890 in 2021, \$17,650 in 2020, and \$17,000 in 2000. Ten percent of households make less than this level.

In [3]:
# Median Household income by race/ethnicity
url = ('https://www2.census.gov/programs-surveys/'+
       'demo/tables/p60/279/tableA2.xlsx')
df = pd.read_excel(url, header=4, index_col=0, skipfooter=34)
grps = [('Black', 'BLACK ALONE 27', 'ASIAN ALONE OR IN COMBINATION', 
         'BLACK 26'),
        ('Asian', 'ASIAN ALONE 28', 'AMERICAN INDIAN AND ALASKA NATIVE ALONE OR IN COMBINATION', 
         'ASIAN AND PACIFIC ISLANDER 26'),
        ('White', 'WHITE ALONE, NOT HISPANIC 25', 
         'BLACK ALONE OR IN COMBINATION', 'WHITE, NOT HISPANIC 26'),
        ('Hispanic', 'HISPANIC (ANY RACE) 30', '', '')]
data = (df['Estimate'].reset_index().fillna(method='ffill', axis=1)
        .set_index('index'))
res = pd.DataFrame()
for name, start, end, drop in grps:
    if end == '':
        s = (data.loc[start: ].iloc[1:-1]).Estimate
    else:
        s = (data.loc[start: end].iloc[1:-1].drop(drop)).Estimate
    s.index = (pd.to_datetime(s.index.astype('str').str[:4]) + 
               pd.DateOffset(months=6))
    loc1 = np.where(s.index.get_loc('2017-07-01'))[0][1]
    loc2 = np.where(s.index.get_loc('2013-07-01'))[0][1]
    res = res.join(s.iloc[:loc1].rename(f'{name}_LT'), how='outer')
    res = res.join(s.iloc[loc1: loc2].rename(f'{name}_PR'), how='outer')
    res = res.join(s.iloc[loc2:].rename(f'{name}_PR2'), how='outer')
    
res.loc['1989':].divide(1000).to_csv(data_dir / 'real_med_hh_inc.csv', 
                                     index_label='date')

cols = [('Wh', 'green!80!black'), ('Bl', 'blue!80!black'), 
        ('Hi', 'red!90!black'), ('As', 'orange')]
cl = {name: c_line(color) for name, color in cols}
ltdt = res.index[-1].year
write_txt(text_dir / 'rmhi_ltdt.txt', f'{ltdt}')
prdt = res.index[-2].year
ltval = res.iloc[-1].dropna().apply(lambda x: f'\${x:,.0f}')
prval = res.iloc[-2].dropna().apply(lambda x: f'\${x:,.0f}')
text = (f'Black median household income is {ltval.Black_LT} in '+
        f'{ltdt}, compared to an inflation-adjusted equivalent '+
        f'of {prval.Black_LT} in {prdt} {cl["Bl"]}. Non-Hispanic '+
        f'white median household income is {ltval.White_LT} in '+
        f'{ltdt} and {prval.White_LT} in {prdt} {cl["Wh"]}. '+
        f'Hispanic (any race) median household income is '+
        f'{ltval.Hispanic_LT} in {ltdt} and {prval.Hispanic_LT} '+
        f'in {prdt} {cl["Hi"]}. Asian median household income is '+
        f'{ltval.Asian_LT} in {ltdt} and {prval.Asian_LT} in '+
        f'{prdt} {cl["As"]}. ')
write_txt(text_dir / 'real_med_hh_inc.txt', text)
print(text)

Black median household income is \$52,860 in 2022, compared to an inflation-adjusted equivalent of \$52,080 in 2021 (see {\color{blue!80!black}\textbf{---}}). Non-Hispanic white median household income is \$81,060 in 2022 and \$84,110 in 2021 (see {\color{green!80!black}\textbf{---}}). Hispanic (any race) median household income is \$62,800 in 2022 and \$62,520 in 2021 (see {\color{red!90!black}\textbf{---}}). Asian median household income is \$108,700 in 2022 and \$109,400 in 2021 (see {\color{orange}\textbf{---}}). 


### Full-time, full-year workers

In [9]:
ltyear = 2023
pry, lty = [str(ltyear-1), str(ltyear)]
res = {}
for year in [pry, lty]:
    res[year] = {}
    varlist = ['WEWKRS', 'MARSUPWT', 'PRECORD']
    dictfile = f'{path}/persfmt{year[2:]}.txt'
    dd = open(dictfile).read()
    p = f'({"|".join(varlist)})\s+(\d+)\s+(\d+)\s'
    cols = {name: (int(start) - 1, int(start) - 1 + int(length)) 
            for name, length, start in re.findall(p, dd)}

    datafile = f'{path}/asec{year}_pubuse.dat'
    df = (pd.read_fwf(datafile, 
                      colspecs=list(cols.values()), 
                      header=None, 
                      names=list(cols.keys()))
            .query('PRECORD == 3'))
    
    data = df.groupby('WEWKRS').MARSUPWT.sum() / 100_000_000
    res[year]['tot'] = data[1]
    res[year]['sh'] = data[1] / data.sum() * 100

text = (f'{res[lty]["tot"]:.1f} million fully-employed people in '+
        f'{ltyear - 1}, equivalent to {res[lty]["sh"]:.1f} percent '+
        f'of the US population, compared to {res[pry]["tot"]:.1f} '+
        f'million ({res[pry]["sh"]:.1f} percent) in {ltyear - 2}. ')
write_txt(text_dir / 'asec_ftfy_tot.txt', text)
print(text)

121.4 million fully-employed people in 2022, equivalent to 36.7 percent of the US population, compared to 117.4 million (35.7 percent) in 2021. 


### Poverty Rates (SPM and OPM)

In [53]:
# Historical SPM
#url = ('https://www.povertycenter.columbia.edu/s/'+
#       'Anchored-and-Historical-Poverty-Rates-1967-to-2020.xlsx')
data = [(1989, 14.3), (1990, 14.8), (1991, 15.2), (1992, 15.8),
        (1993, 16.8), (1994, 15.7), (1995, 14.2), (1996, 14.1), 
        (1997, 13.6), (1998, 13.0), (1999, 12.4), (2000, 12.4), 
        (2001, 13.2), (2002, 13.6), (2003, 13.8), (2004, 13.7), 
        (2005, 13.7), (2006, 13.8), (2007, 14.4), (2008, 14.8)]
idx, values = zip(*data)

spmh = pd.Series(values, idx).rename('PR4')
spmh.index = (pd.to_datetime(spmh.index.astype('str')) + 
              pd.DateOffset(months=6))

In [65]:
loc1 = np.where(s.index == '2013-07-01')[0][1]
loc2 = np.where(s.index == '2017-07-01')[0][1]
res = pd.DataFrame()
res = res.join(s.iloc[:loc2].rename({'Percent': 'OPM_LT'}, axis=1), how='outer')
res = res.join(s.iloc[loc2: loc1].rename({'Percent': 'OPM_PR'}, axis=1), how='outer')
res = res.join(s.iloc[loc1:].rename({'Percent': 'OPM_PR2'}, axis=1), how='outer')
    
res.loc['1989':].to_csv(data_dir / 'poverty_opm_line.csv', 
                                     index_label='date')
opmcol = 'green!70!blue'
node = end_node(res.OPM_LT, opmcol, offset=-0.06)
write_txt(text_dir / 'poverty_opm_node.txt', node)

#SPM manual
spm = {'PR4': {'2009': 15.1, '2010': 16.0, '2011': 16.1, 
               '2012': 16.0, '2013': 15.5},
       'PR3': {'2013': 15.8, '2014': 15.3, '2015': 14.3},
       'PR2': {'2015': 14.5, '2016': 14.0, '2017': 13.9},
       'PR': {'2017': 13.0, '2018': 12.8, '2019': 11.7},
       'LT': {'2019': 11.8, '2020': 9.2, '2021': 7.8, 
              '2022': 12.4}}

sp = pd.DataFrame(spm)
sp.index = (pd.to_datetime(sp.index.astype('str').str[:4]) + 
               pd.DateOffset(months=6))
sp = pd.concat([spmh.to_frame(), sp])  
sp.to_csv(data_dir / 'poverty_spm_line.csv', index_label='date')
spmcol = 'violet' 
node = end_node(sp.LT, spmcol, date='y', 
                full_year=True, offset=0.2)
write_txt(text_dir / 'poverty_spm_node.txt', node)

maxval = res.loc['1989':].max().max()
minval = res.loc['1989':].min().min()
avgval = res.loc['1989':].mean().mean()
spm19 = sp.loc['2019-07-01', 'LT']
opm19 = res.loc['2019-07-01', 'OPM_LT']
spm21 = sp.loc['2021-07-01', 'LT']
opm21 = res.loc['2021-07-01', 'OPM_LT']
spm22 = sp.loc['2022-07-01', 'LT']

text = (f'Since 1989, the official poverty measure {c_line(opmcol)} '+
        f'shows between {minval} percent and {maxval} percent of '+
        f'people in poverty each year, with an average poverty rate of '+
        f'{avgval:.1f} percent during the period. Poverty rates '+
        'were above-average after the recession of 1991 and after '+
        'the great recession, and below-average around 2000.')
write_txt(text_dir / 'poverty_rate_time.txt', text)
print(text, '\n')  
see = c_line(spmcol).replace('(', '(SPM, ')
text2 = ('In 2019, both the official US poverty rate '+
         'and the more-comprehensive supplemental rate '+
         f'{see} reached new lows of {opm19} '+
         f'percent and {spm19} percent, respectively.\n\n'
         f'In 2021, the official rate increased to {opm21} percent, '+
         f'while the SPM fell further, '+
         f'to a new low of {spm21} percent. '+
         'The official poverty rate does not include stimulus checks, '+
         'housing assistance, or tax credits, while the supplemental '+
         'rate does. In 2022, the SPM bounced back to '+
         f'{spm22} percent, as stimulus expired.')
write_txt(text_dir / 'poverty_rate_time2.txt', text2)
print(text2)

Since 1989, the official poverty measure (see {\color{green!70!blue}\textbf{---}}) shows between 10.5 percent and 15.1 percent of people in poverty each year, with an average poverty rate of 12.9 percent during the period. Poverty rates were above-average after the recession of 1991 and after the great recession, and below-average around 2000. 

In 2019, both the official US poverty rate and the more-comprehensive supplemental rate (SPM, see {\color{violet}\textbf{---}}) reached new lows of 10.5 percent and 11.8 percent, respectively.

In 2021, the official rate increased to 11.6 percent, while the SPM fell further, to a new low of 7.8 percent. The official poverty rate does not include stimulus checks, housing assistance, or tax credits, while the supplemental rate does. In 2022, the SPM bounced back to 12.4 percent, as stimulus expired.


### Who are the poor?

Attempt to replicate excellent chart by Matt Bruenig

----

Source: CPS ASEC

In [3]:
year = 2022
write_txt(text_dir / 'poverty_ltyear.txt', f'{year}')

# Extract person records for selected variables
variables = ['H_SEQ', 'PPPOS', 
             'PRECORD', 'A_AGE', 'MARSUPWT', 'PERLIS', 
             'SPM_Resources', 'SPM_PovThreshold', 'WKSWORK',
             'PRDISFLG', 'RSNNOTW', 'PYRSN', 'LKWEEKS',
             'SPM_Weight', 'SPM_ID', 'SPM_CapWkCCXpns',
             'SPM_Totval', 'SPM_MedXpns', 'SPM_ChildSupPd',
             'SPM_Poor', 'SPM_NumPer', 'SPM_EIP']

benefits = ['SS_VAL', 'SSI_VAL', 'UC_VAL', 'VET_VAL', 
            'WC_VAL', 'PAW_VAL']

dictfile = f'{path}/persfmt23.txt'
dd = open(dictfile).read()
p = f'({"|".join(variables + benefits)})\s+(\d+)\s+(\d+)\s'
cols = {name: (int(start) - 1, int(start) - 1 + int(length)) 
        for name, length, start in re.findall(p, dd)}

datafile = f'{path}/asec2023_pubuse.dat'
df = (pd.read_fwf(datafile, 
                  colspecs=list(cols.values()), 
                  header=None, 
                  names=list(cols.keys()))
        .query('PRECORD == 3'))

In [3]:
# Categorize population
df['Category'] = (np.where(df.A_AGE < 18, 'Children', 
                  np.where(df.A_AGE > 64, 'Elderly', 
                  np.where(((df.PRDISFLG == 1) | (df.PYRSN == 1) | (df.RSNNOTW == 1)), 'Disabled', 
                  np.where(((df.PYRSN == 3) | (df.RSNNOTW == 4)), 'Students', 
                  np.where(((df.PYRSN == 2) | (df.RSNNOTW == 3)), 'Carers',
                  np.where(((df.PYRSN == 5) | (df.RSNNOTW == 5) | (df.LKWEEKS > 0)), 'Unemployed',         
                  np.where(((df.PYRSN == 4) | (df.RSNNOTW == 2)), 'Early \\\\ Retired',  
                  np.where(df.WKSWORK > 49, 'Fully \\\\ Employed', 'All Other')))))))))

df['AGE_GRP'] = pd.cut(df.A_AGE, range(0, 79, 3))

df['SPM'] = np.where(df['SPM_Resources'] < df['SPM_PovThreshold'], 1, 0)
df['OPM'] = np.where(df['PERLIS'] == 1, 1, 0)
df['MARKET_INCOME'] = (df['SPM_Totval'] - 
                       df[['SPM_CapWkCCXpns','SPM_MedXpns', 'SPM_ChildSupPd']].sum(axis=1) - 
                       df[benefits].sum(axis=1).groupby(df['SPM_ID']).transform('sum'))

df['SPM_MI'] = np.where(df['MARKET_INCOME'] < df['SPM_PovThreshold'], 1, 0)
df['MI_GAP'] = ((df['SPM_PovThreshold'] - df['MARKET_INCOME']) / 
                df['SPM_NumPer']) * df['SPM_Weight'] / 100
df['SPM_GAP'] = ((df['SPM_PovThreshold'] - df['SPM_Resources']) / 
                 df['SPM_NumPer']) * df['SPM_Weight'] / 100

### Total Amount of Poverty

In [4]:
spmtot = df.query('SPM == 1').SPM_GAP.sum() / 1_000_000
mi_tot = df.query('SPM_MI == 1').MI_GAP.sum() / 1_000_000

### Who are the poor

In [5]:
# Only below poverty line
data = df.query('SPM == 1')
data2 = df.query('OPM == 1')
data3 = df.query('SPM_MI == 1')

results = pd.DataFrame()
# Group share of total
results['SPM'] = (data.groupby('Category').SPM_Weight.sum() / 
                  data.SPM_Weight.sum() * 100).sort_values(ascending=False)
results['OPM'] = (data2.groupby('Category').MARSUPWT.sum() / 
                  data2.MARSUPWT.sum() * 100).sort_values(ascending=False)
results['SPM_MI'] = (data3.groupby('Category').SPM_Weight.sum() / 
                  data3.SPM_Weight.sum() * 100).sort_values(ascending=False)
results.to_csv(data_dir / 'poor.csv', index_label='name')

In [6]:
results2 = pd.DataFrame()
# Poverty rate of each group
results2['SPM'] = (df.groupby('Category')
                     .apply(lambda x: np.average(x['SPM'], weights=x['SPM_Weight']))
                     * 100).loc[results.index]
results2['OPM'] = (df.groupby('Category')
                     .apply(lambda x: np.average(x['OPM'], weights=x['MARSUPWT']))
                     * 100).loc[results.index]
results2['SPM_MI'] = (df.groupby('Category')
                     .apply(lambda x: np.average(x['SPM_MI'], weights=x['SPM_Weight']))
                     * 100).loc[results.index]
results2.to_csv(data_dir / 'poor2.csv', index_label='name')

In [7]:
results3 = pd.DataFrame()
results3['disposable'] = (data.groupby('AGE_GRP')['SPM_GAP'].sum() / 3e9)
results3['market'] = (data3.groupby('AGE_GRP')['MI_GAP'].sum() / 3e9) - results3['disposable']

results3.index = [i.left for i in results3.index]
results3.to_csv(data_dir / 'poor_age_latest.csv', index_label='age')
#results3.iloc[::-1].plot(kind='area', stacked=True, figsize=(8,8), color=['limegreen', 'royalblue']);

In [8]:
opm_tot = df.query('OPM == 1').MARSUPWT.sum() / 1_000_000 / 100
mi_tot = df.query('SPM_MI == 1').MARSUPWT.sum() / 1_000_000 / 100
mi_rt = np.average(df['SPM_MI'], weights=df['MARSUPWT']) * 100
spm_tot = df.query('SPM == 1').MARSUPWT.sum() / 1_000_000 / 100
spm_rt = np.average(df['SPM'], weights=df['MARSUPWT']) * 100
ch_val = mi_tot - spm_tot

url = 'https://www.census.gov/library/publications/2022/demo/p60-277.html'
text = (f'In {year}, market income is below the poverty line for '+
        f'about one quarter ({mi_rt:.1f} percent) of US households. Government '+
        f'programs reduce the poverty rate to {spm_rt:.1f} percent, '+
        f'\href{{{url}}}{{pulling}} {ch_val:.1f} million people out of poverty '+
        f'in {year}. ')
write_txt(text_dir / 'poverty_intro1.txt', text)
print(text, '\n')

text = (f'Under this definition, {opm_tot:.1f} million people are in poverty '+
        f'in the US in {year}. ')
write_txt(text_dir / 'poverty_intro2.txt', text)
print(text, '\n')

text = (f'The SPM poverty level in {year} is {spm_tot:.1f} million people.')
write_txt(text_dir / 'poverty_intro3.txt', text)
print(text)

groups = ['Children', 'Elderly', 'Disabled', 'Carers', 'Students']
gs_pop = (df.groupby('Category')['MARSUPWT'].sum() / 
          df['MARSUPWT'].sum())[groups].sum() * 100
gs_pop_round = 5*round(gs_pop/5)
padj = ('nearly ' if gs_pop_round > (gs_pop + 1) 
        else 'more than ' if (gs_pop_round + 1) > gs_pop 
        else 'roughly ')
group_share = results.loc[groups, 'SPM'].sum()
gs_mi = results.loc[groups, 'SPM_MI'].sum()
gs_spm_round = 5*round(group_share/5)
gs_mi_round = round(gs_mi,-1)

rmiadj = ('nearly ' if gs_mi_round > (gs_mi + 2) 
        else 'more than ' if (gs_mi_round + 2) < gs_mi 
        else 'roughly ')
text = (f'These groups make up {padj}{gs_pop_round} percent of the population '+
        f'but {rmiadj}{gs_mi_round:.0f} percent of those in poverty before taxes '+
        'and transfers.')
write_txt(text_dir / 'poverty_groups.txt', text)
print(text)

In 2022, market income is below the poverty line for about one quarter (23.7 percent) of US households. Government programs reduce the poverty rate to 12.4 percent, \href{https://www.census.gov/library/publications/2022/demo/p60-277.html}{pulling} 37.4 million people out of poverty in 2022.  

Under this definition, 37.9 million people are in poverty in the US in 2022.  

The SPM poverty level in 2022 is 41.0 million people.
These groups make up roughly 50 percent of the population but roughly 80 percent of those in poverty before taxes and transfers.


### 2014 Data for comparison

In [9]:
spm = pd.read_stata(f'{path}/spmresearch2014.dta')

In [10]:
dictfile = f'{path}/asec2015early_pubuse.dd.txt'
dd = open(dictfile, encoding='iso-8859-1').read()
p = re.compile(f'D ({"|".join(variables + benefits)})\s+(\d+)\s+(\d+)\s+')
cols = {name: (int(start) - 1, int(start) - 1 + int(length)) 
        for name, length, start in re.findall(p, dd)}

datafile = f'{path}/asec2015_pubuse.dat'
df = (pd.read_fwf(datafile, 
                  colspecs=list(cols.values()), 
                  header=None, 
                  names=list(cols.keys()))
        .query('PRECORD == 3').astype('int'))

In [11]:
df = pd.merge(spm, df, left_on=['h_seq', 'pppos'], right_on=['H_SEQ', 'PPPOS'])

# Categorize population
df['Category'] = (np.where(df.A_AGE < 18, 'Children', 
                  np.where(df.A_AGE > 64, 'Elderly', 
                  np.where(((df.PRDISFLG == 1) | (df.PYRSN == 1) | (df.RSNNOTW == 1)), 'Disabled', 
                  np.where(((df.PYRSN == 3) | (df.RSNNOTW == 4)), 'Students', 
                  np.where(((df.PYRSN == 2) | (df.RSNNOTW == 3)), 'Carers',
                  np.where(((df.PYRSN == 5) | (df.RSNNOTW == 5) | (df.LKWEEKS > 0)), 'Unemployed',         
                  np.where(((df.PYRSN == 4) | (df.RSNNOTW == 2)), 'Early Retired',  
                  np.where(df.WKSWORK > 49, 'Fully Employed', 'All Other')))))))))

df['AGE_GRP'] = pd.cut(df.A_AGE, range(0, 79, 3))

df['SPM'] = np.where(df['spmu_poor'] == 1, 1, 0)
df['OPM'] = np.where(df['PERLIS'] == 1, 1, 0)
df['MARKET_INCOME'] = (df['spmu_totval'] - 
                       df[['spmu_capwknchcarexpns','spmu_medoopnmcareb', 'spmu_childsuppd']].sum(axis=1) - 
                       df[benefits].sum(axis=1).groupby(df['spmu_id']).transform('sum'))

df['SPM_MI'] = np.where(df['MARKET_INCOME'] < df['spmu_povthreshold'], 1, 0)

df['MI_GAP'] = ((df['spmu_povthreshold'] - df['MARKET_INCOME']) / df['spmu_numper']) * df['spmu_weight'] / 100
df['SPM_GAP'] = ((df['spmu_povthreshold'] - df['spmu_resources']) / df['spmu_numper']) * df['spmu_weight'] / 100

In [12]:
# Only below poverty line
data = df.query('SPM == 1')
data2 = df.query('OPM == 1')
data3 = df.query('SPM_MI == 1')

results = pd.DataFrame()
# Group share of total
results['SPM'] = (data.groupby('Category').spmu_weight.sum() / 
                  data.spmu_weight.sum() * 100).sort_values(ascending=False)
results['OPM'] = (data2.groupby('Category').MARSUPWT.sum() / 
                  data2.MARSUPWT.sum() * 100).sort_values(ascending=False)
results['SPM_MI'] = (data3.groupby('Category').spmu_weight.sum() / 
                  data3.spmu_weight.sum() * 100).sort_values(ascending=False)
#print(results)

In [13]:
results2 = pd.DataFrame()
# Poverty rate of each group
results2['SPM'] = (df.groupby('Category')
                     .apply(lambda x: np.average(x['SPM'], weights=x['spmu_weight']))
                     * 100).loc[results.index]
results2['OPM'] = (df.groupby('Category')
                     .apply(lambda x: np.average(x['OPM'], weights=x['MARSUPWT']))
                     * 100).loc[results.index]
results2['SPM_MI'] = (df.groupby('Category')
                     .apply(lambda x: np.average(x['SPM_MI'], weights=x['spmu_weight']))
                     * 100).loc[results.index]
#print(results2)

#### Price adjustment
https://www.bls.gov/cpi/research-series/r-cpi-u-rs-allitems.pdf

In [14]:
results3 = pd.DataFrame()
results3['disposable'] = (data.groupby('AGE_GRP')['SPM_GAP'].sum() / 3e9)# - results3['ca_gap']
results3['market'] = (data3.groupby('AGE_GRP')['MI_GAP'].sum() / 3e9) - results3['disposable']# - results3['ca_gap']
cpi = pd.read_csv(data_dir / 'cpi_u_rs.csv', index_col='date', parse_dates=True)
results3 = (results3 * cpi.loc['2022-07-01', 'value'] / cpi.loc['2014-07-01', 'value']) # CPI-U-RS
results3.index = [i.left for i in results3.index]
results3.to_csv(data_dir / 'poor_age_prev.csv', index_label='age')
#results3.iloc[::-1].plot(kind='area', stacked=True, figsize=(8,8), color=['limegreen', 'royalblue']);